In [1]:
import pandas as pd
import numpy as np

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import *
import random as sysrandom
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook as tqdm
from tqdm import trange
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
from keras.preprocessing.sequence import pad_sequences

%pylab inline
%load_ext tensorboard

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [3]:
dfa = pd.read_csv('datasets/deepmatcher/walmart-amazon-dirty/tableA.csv')
dfb = pd.read_csv('datasets/deepmatcher/walmart-amazon-dirty/tableB.csv')
train = pd.read_csv('datasets/deepmatcher/walmart-amazon-dirty/train.csv')
valid = pd.read_csv('datasets/deepmatcher/walmart-amazon-dirty/valid.csv')
test = pd.read_csv('datasets/deepmatcher/walmart-amazon-dirty/test.csv')

# dfa = pd.read_csv('datasets/deepmatcher/company/tableA.csv')
# dfb = pd.read_csv('datasets/deepmatcher/company/tableB.csv')
# train = pd.read_csv('datasets/deepmatcher/company/train.csv')
# valid = pd.read_csv('datasets/deepmatcher/company/valid.csv')
# test = pd.read_csv('datasets/deepmatcher/company/test.csv')

In [5]:
train_tuples = []
for i,row in tqdm(train.iterrows()):
    train_tuples.append(
                    (str(dfa[dfa.id == row.ltable_id].title.values[0]) + str(dfa[dfa.id == row.ltable_id].category.values[0]) + str(dfa[dfa.id == row.ltable_id].brand.values[0]) + str(dfa[dfa.id == row.ltable_id].modelno.values[0]) + str(dfa[dfa.id == row.ltable_id].price.values[0]),
                         str(dfb[dfb.id == row.rtable_id].title.values[0]) + str(dfb[dfb.id == row.rtable_id].category.values[0]) + str(dfb[dfb.id == row.rtable_id].brand.values[0]) + str(dfb[dfb.id == row.rtable_id].modelno.values[0]) + str(dfb[dfb.id == row.rtable_id].price.values[0]),
                          row.label
                       )
    )
train_df = pd.DataFrame(train_tuples, columns=["review1","review2","label"])

validation_tuples = []
for i,row in tqdm(valid.iterrows()):
    validation_tuples.append(
                    (str(dfa[dfa.id == row.ltable_id].title.values[0]) + str(dfa[dfa.id == row.ltable_id].category.values[0]) + str(dfa[dfa.id == row.ltable_id].brand.values[0]) + str(dfa[dfa.id == row.ltable_id].modelno.values[0]) + str(dfa[dfa.id == row.ltable_id].price.values[0]),
                         str(dfb[dfb.id == row.rtable_id].title.values[0]) + str(dfb[dfb.id == row.rtable_id].category.values[0]) + str(dfb[dfb.id == row.rtable_id].brand.values[0]) + str(dfb[dfb.id == row.rtable_id].modelno.values[0]) + str(dfb[dfb.id == row.rtable_id].price.values[0]),
                          row.label
                       )
    )
val_df = pd.DataFrame(validation_tuples, columns=["review1","review2","label"])

test_tuples = []
for i,row in tqdm(test.iterrows()):
    test_tuples.append(
                    (str(dfa[dfa.id == row.ltable_id].title.values[0]) + str(dfa[dfa.id == row.ltable_id].category.values[0]) + str(dfa[dfa.id == row.ltable_id].brand.values[0]) + str(dfa[dfa.id == row.ltable_id].modelno.values[0]) + str(dfa[dfa.id == row.ltable_id].price.values[0]),
                         str(dfb[dfb.id == row.rtable_id].title.values[0]) + str(dfb[dfb.id == row.rtable_id].category.values[0]) + str(dfb[dfb.id == row.rtable_id].brand.values[0]) + str(dfb[dfb.id == row.rtable_id].modelno.values[0]) + str(dfb[dfb.id == row.rtable_id].price.values[0]),
                          row.label
                       )
    )
test_df = pd.DataFrame(test_tuples, columns=["review1","review2","label"])

In [6]:
train_df.sample(20)

,review1,review2,label
4736,sandisk 8gb cruzer usb flash drive pink usb dr...,sandisk cruzer blade 16gb usb 2.0 flash drive ...,0
5821,da-lite da-glas standard rear projection scree...,da-lite 27565 da-plex deluxe rear projection s...,0
1339,western digital wd scorpio blue 320gb 5400rpm ...,western digital wd1600bevt 160gb 2.5 sata 5400...,0
4509,us brown bear fixed series medium slim fixed m...,us brown bear w9-63f-blk slim large fixed moun...,0
397,western digital 2tb my book studio edition ii ...,western digital wd elements 2 tb usb 2.0 deskt...,0
1752,da-lite da-plex thru-the-wall rear projection ...,da-lite 27597 da-glas unframed rear projection...,0
2688,startech 2.5 in superspeed usb 3.0 sata hard d...,startech.com 2.5-inch silver usb 2.0 external ...,0
6114,elmer s 4-count washable school glue sticks - ...,elmer s e308 - washable school glue 7.62 oz li...,0
2668,griffin technology screen care kit for ipadmp3...,griffin technology reveal for ipad - black 9.4...,0
5738,veho vcca002wpc waterproof case for muvi muvit...,veho vcca002wpc waterproof case for muvi micro...,1


In [7]:
cdf = pd.concat([train_df,val_df])
cdf.reset_index(drop=True, inplace=True)
train_indices = list(range(len(train_df)))

In [8]:
val_indices = list(range(len(train_df), len(cdf)))

In [9]:
subdf_siamese = cdf

In [10]:
subdf_siamese["review"] = subdf_siamese.review1.map(str) + " [SEP] " + subdf_siamese.review2.map(str)

In [11]:
class ReviewsDataLoader:
    
    def __init__(self, df, split_ratio=0.8, max_length = 200, batchsize = 16, train_indices = None, val_indices = None):
        self.df = df
        self.ratio = split_ratio
        self.lb = LabelEncoder()
        self.maxlen = max_length
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        self.attention_masks = []
        self.batch_size = batchsize
        self.train_indices = train_indices
        self.val_indices = val_indices
        self.processInput()
        self.splitData()
        self.size = len(self.df)
        
    def processInput(self):
        self.labels = self.lb.fit_transform(self.df.label)
        sentences = self.df.review.values
        sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
        tokenized_texts = [self.tokenizer.tokenize(sent) for sent in sentences]
        input_ids = [self.tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
        self.input_ids = pad_sequences(input_ids, maxlen=self.maxlen, dtype="long", truncating="post", padding="post")
        for seq in self.input_ids:
            seq_mask = [float(i>0) for i in seq]
            self.attention_masks.append(seq_mask)
        self.attention_masks = np.array(self.attention_masks)
        
    def splitData(self):
        if self.train_indices and self.val_indices:
            self.train_inputs = np.array(self.input_ids)[self.train_indices]
            self.validation_inputs = np.array(self.input_ids)[self.val_indices]
            self.train_labels = np.array(self.labels)[self.train_indices]
            self.validation_labels = np.array(self.labels)[self.val_indices]
            
            self.train_masks = np.array(self.attention_masks)[self.train_indices]
            self.validation_masks = np.array(self.attention_masks)[self.val_indices]
        else:   
            self.train_inputs, self.validation_inputs, self.train_labels, self.validation_labels = train_test_split(self.input_ids, self.labels, 
                                                                random_state=2018, train_size=self.ratio)
            self.train_masks, self.validation_masks, _, _ = train_test_split(self.attention_masks, self.input_ids,
                                                 random_state=2018, train_size=self.ratio)


    def getDataloaders(self):
        self.train_inputs = torch.tensor(self.train_inputs)
        self.validation_inputs = torch.tensor(self.validation_inputs)
        self.train_labels = torch.tensor(self.train_labels)
        self.validation_labels = torch.tensor(self.validation_labels)
        self.train_masks = torch.tensor(self.train_masks)
        self.validation_masks = torch.tensor(self.validation_masks)
        
        train_data = TensorDataset(self.train_inputs, self.train_masks, self.train_labels)
        train_sampler = RandomSampler(train_data)
        self.train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=self.batch_size)

        validation_data = TensorDataset(self.validation_inputs, self.validation_masks, self.validation_labels)
        validation_sampler = SequentialSampler(validation_data)
        self.validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=self.batch_size)
        return self.train_dataloader, self.validation_dataloader


In [12]:
#rdl = ReviewsDataLoader(subdf)
rdl = ReviewsDataLoader(subdf_siamese, batchsize=64, max_length=100, split_ratio=0.9, train_indices=train_indices, val_indices=val_indices)

In [13]:
train_dataloader, validation_dataloader = rdl.getDataloaders()

In [15]:
rdl.train_inputs[0]

tensor([  101,  7069, 12117, 25022,  2638,  2860, 16584,  2063,  5988, 21926,
         2629,  2186,  4964,  4853,  2898,  3898,  1011,  5594, 19754,  8139,
         1011,  2236,  7069, 12117,  1054, 27531,  2860,  2232,  2487,  1011,
         2898,  2871,  2683,  1012,  1014,  7229,  7229,  7229,  7229,   102,
        25022,  2638, 17643,  2100,  1041,  2480, 15643,  2186,  4964,  4853,
         3898,  1011,  5018, 19754,  6584,  2683,  1012,  1014, 21572, 20614,
         3258, 12117, 20806,  6528, 25789,  2078,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter(log_dir='logs/deepmatcher_walmart-amazon_cnn')
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")

Let's use 3 GPUs!


In [18]:
class BertForSequenceClassificationSiamese(nn.Module):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.
    Params:
        `config`: a BertConfig class instance with the configuration to build a new model.
        `num_labels`: the number of classes for the classifier. Default = 2.
    Inputs:
        `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length]
            with the word token indices in the vocabulary. Items in the batch should begin with the special "CLS" token. (see the tokens preprocessing logic in the scripts
            `extract_features.py`, `run_classifier.py` and `run_squad.py`)
        `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length] with the token
            types indices selected in [0, 1]. Type 0 corresponds to a `sentence A` and type 1 corresponds to
            a `sentence B` token (see BERT paper for more details).
        `attention_mask`: an optional torch.LongTensor of shape [batch_size, sequence_length] with indices
            selected in [0, 1]. It's a mask to be used if the input sequence length is smaller than the max
            input sequence length in the current batch. It's the mask that we typically use for attention when
            a batch has varying length sentences.
        `labels`: labels for the classification output: torch.LongTensor of shape [batch_size]
            with indices selected in [0, ..., num_labels].
    Outputs:
        if `labels` is not `None`:
            Outputs the CrossEntropy classification loss of the output with the labels.
        if `labels` is `None`:
            Outputs the classification logits of shape [batch_size, num_labels].
    Example usage:
    ```python
    # Already been converted into WordPiece token ids
    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
    num_labels = 2
    model = BertForSequenceClassification(config, num_labels)
    logits = model(input_ids, token_type_ids, input_mask)
    ```
    """
    def __init__(self, num_labels=2): # Change number of labels here.
        super(BertForSequenceClassificationSiamese, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768*2, num_labels)
        #self.fc1 = nn.Linear(config.hidden_size*2, 512)
        #nn.init.xavier_normal_(self.classifier.weight)

    '''def forward_once(self, x):
        # Forward pass
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output'''

    def forward_once(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask) #, output_all_encoded_layers=False
        pooled_output = self.dropout(pooled_output)
        #logits = self.classifier(pooled_output)

        return pooled_output
    
    def forward(self, input_ids1, input_ids2, labels=None ):
        # forward pass of input 1
        output1 = self.forward_once(input_ids1, token_type_ids=None, attention_mask=None, labels=labels)
        # forward pass of input 2
        output2 = self.forward_once(input_ids2, token_type_ids=None, attention_mask=None, labels=labels)
        
        ## We only need to return the embeddings for contrastive loss
        out = torch.cat((output1, output2), 1)
        #print(out.shape)

        #out = self.fc1(out)
        logits = self.classifier(out)

        return logits

    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False

    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [19]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [20]:
#criterion = ContrastiveLoss()
criterion = nn.CrossEntropyLoss()

In [21]:
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=50)
# model = model.to(device)
# model_parallel = nn.DataParallel(model)

In [22]:
model_siamese = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [23]:
model_siamese = model_siamese.to(device)
model_siamese_parallel = nn.DataParallel(model_siamese)

In [24]:
param_optimizer = list(model_siamese_parallel.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [25]:
import os
from sklearn.metrics import f1_score, precision_score, recall_score
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_f1(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(pred_flat, labels_flat)

def checkpoint(model, acc, acc_array, path="models", name="yelp_dataset_50"):
    if not os.path.exists(path):
        os.makedirs(path)
    if acc_array:
        if acc > max(acc_array):
            torch.save(model.state_dict(), os.path.join(path,"{}_{}".format(name,acc)))
    else:
        torch.save(model.state_dict(), os.path.join(path,"{}_{}".format(name,acc)))
        
def checkpoint_abs(model, path="models", name="yelp_dataset_50"):
    torch.save(model.state_dict(), os.path.join(path,"{}".format(name)))

In [26]:
global_epoch = 1
#%%capture output
train_loss_set = []
val_loss_set = []
val_accuracies = []

train_epochs = 50
t_total = len(train_dataloader) // 1 * train_epochs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)
# optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=1e-8)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=0, t_total=t_total)

# Number of training epochs (authors recommend between 2 and 4)

# trange is a tqdm wrapper around the normal python range
#for t in trange(train_epochs, desc="Epoch"):
for _ in range(train_epochs):

  # Training 
  # Set our model to training mode (as opposed to evaluation mode)
    
  model_siamese_parallel.train()
  tr_loss = 0
  val_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(tqdm(train_dataloader)):
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model_siamese_parallel(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = loss[0].mean()
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {:.3f}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model_siamese_parallel.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  eval_f1 = 0
  nb_eval_steps, nb_eval_examples = 0, 0
  pred_arr = []
  label_arr = []

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model_siamese_parallel(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
      # Move logits and labels to CPU
      logits = logits[0].detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
      tmp_eval_f1 = flat_f1(logits, label_ids)

      eval_accuracy += tmp_eval_accuracy
      eval_f1 += tmp_eval_f1
      nb_eval_steps += 1
        
      pred_arr += list(np.argmax(logits, axis=1).flatten())
      label_arr += list(label_ids.flatten())
  
  print(label_arr,"\n\n",pred_arr)

  print("Validation Accuracy: {:.3f}".format(eval_accuracy/nb_eval_steps))
  #print("Validation F1: {:.3f}".format(eval_f1/nb_eval_steps))
  print("Validation F1: {:.3f}".format(f1_score(label_arr, pred_arr)))
  print("Validation Precision: {:.3f}".format(precision_score(label_arr, pred_arr)))
  print("Validation Recall: {:.3f}".format(recall_score(label_arr, pred_arr)))
  val_acc = eval_accuracy/nb_eval_steps
#   checkpoint(model_siamese_parallel, val_acc, val_accuracies, 
#              path='models/ontology/', 
#              name="ontology_mapping_original_small")
  val_accuracies.append(val_acc)
  writer.add_scalar('Accuracy/Val', eval_accuracy/nb_eval_steps, global_epoch)
  writer.add_scalar('Loss/Train', tr_loss/nb_tr_steps, global_epoch)
  writer.add_scalar('F1 Score/Val', f1_score(label_arr, pred_arr), global_epoch)
  writer.add_scalar('Precision/Val', precision_score(label_arr, pred_arr), global_epoch)
  writer.add_scalar('Recall/Val', recall_score(label_arr, pred_arr), global_epoch)

  global_epoch += 1
  


/home/awais/.local/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



Train loss: 0.307


/home/awais/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

/home/awais/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/awais/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


...


Train loss: 0.002
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,


Train loss: 0.005
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,


Train loss: 0.004
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,


Train loss: 0.006
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,


Train loss: 0.003
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,